## Data download task for SOC-D

Ever found yourself having to download climate / chemical deposition data and put it in an excel spreadsheet? Well, some people like excel, so in the interests of recording how it was done, the following notebook has been created. 

The data in question is MET office climate vars and deposition data from CEH's very own data repository. (*It is worth noting that the MET office data could be processed on JASMIN, so you could eliminate the downloading aspect of this.*)

The request here was to download the MET and chemical deposition data, attribute a CSS points file with the data, then write this data to an excel spreadsheet where each sheet was seperate variable.

For the underlying processes see [here](https://github.com/Ciaran1981/eot/blob/bdd6e4a75fecd38620af0b8b3cf480a4f94fdcc1/eot/met_tseries.py#L161) (writing the vars to geomnetry), [here ](https://github.com/Ciaran1981/eot/blob/bdd6e4a75fecd38620af0b8b3cf480a4f94fdcc1/eot/met_tseries.py#L241) (writing the MET vars to a sheet) and [here](https://github.com/Ciaran1981/eot/blob/bdd6e4a75fecd38620af0b8b3cf480a4f94fdcc1/eot/met_tseries.py#L306) (writing the deposition vars to a sheet).

This is dependent on a lib of functions I have made which can be found [here](https://github.com/Ciaran1981/eot). Follow the instructions to install. The dependencies used below are all covered by this lib. 


In [2]:
import os 
from eot.downloader import dloadbatch, setup_sesh
import eot.met_tseries as mt
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
from glob import glob
from joblib import Parallel, delayed
from cdo import Cdo
from pyexcelerate import Workbook
from joblib import Parallel, delayed
cdo = Cdo()

### Part 1: MET office data

First, we must setup the session with NEODC to download the data from their server.

In [ ]:
user = "NEODC username"
passw = "NEODC password"

setup_sesh(usr, pss)

Next an input shapefile - you will need to obtain your own - in this instance it was countryside survey points.
This is for later when we attribute each point with both Met climate and CEH deposition data.

In [3]:
inShp = 'path/to/my/cssfile.shp'

We will use this string as a template url from which to create a list of download urls. These get updated, so check on the NEODC for the latest and adjust accordingly.

In [4]:
rain_url16 = ('https://dap.ceda.ac.uk/badc/ukmo-hadobs/data/'
              'insitu/MOHC/HadOBS/HadUK-Grid/v1.1.0.0/1km/hurs/'
              'mon/latest/hurs_hadukgrid_uk_1km_mon_196101-196112.nc')

For the MET vars the time period of interest was 1978-2022.

Some ugly formatting for later processing

In [ ]:
yrs = np.arange(1978, 2022, 1)

years = [str(y)+'01-'+str(y)+'12' for y in yrs]

yrlist = [rain_url16.replace('196101-196112', y) for y in years]

Here we make a directory for everything to go in along with subdirs for each var.

In [ ]:
clim_vars = ['tasmax', 'tasmin', 'tas', 'rainfall', 'sun', 'hurs', 'pv']

os.mkdir('MetDwn78_21')

os.chdir('MetDwn78_21')

clmdirs = [os.mkdir(c) for c in clim_vars if not os.path.isdir(c)]

Now we make a set of input and output paths.

In [ ]:
urlin = []
pthoot = []

for c in clim_vars:
    ins = [y.replace('hurs', c) for y in yrlist]
    oots = [os.path.join(c, os.path.basename(i)) for i in ins]
    urlin.append(ins)
    pthoot.append(oots)

Finally download the files.

This is done in parallel where ```nt``` is the number of threads used.

In [ ]:
nt = len(clim_vars)

_ = Parallel(n_jobs=nt, verbose=2)(delayed(dloadbatch)(u, c) for u, c in zip(urlin, clim_vars))

Now we can merge the individual netcdfs into single files for each var encompassing the whole period (1978-2021). We use the lib ```cdo``` for this purpose as it is fast. This was intialised in the first cell along with the module imports.

```python
cdo = Cdo()
```

In [ ]:
final = []

for p, n in tqdm(zip(pthoot, clim_vars)):
    nm = n+'_78_21.nc'
    ofile = cdo.cat(input=p, output=nm, options='-r')
    final.append(ofile)

Now we can write the data to an excel file. The 'excel workbook' is created using pyexcelerate, which after trying various solutions was the only reliable option.

This is a 2 stage process:
first extracting the time series data to a geo-dataframe,
```python
ndf = mt.met_time_series_to_sheet(f, inShp, n, espgout='epsg:4326')
```
Then writing this data in list form to the excel sheets.

```python
values = [ndf.columns] + list(ndf.values)
ws = wb.new_sheet(n, data=values)
```

In [ ]:
wb = Workbook()

for f, n in tqdm(zip(final, clim_vars)):
    ndf = mt.met_time_series_to_sheet(f, inShp, n, espgout='epsg:4326')
    values = [ndf.columns] + list(ndf.values)
    ws = wb.new_sheet(n, data=values)

wb.save("SOCD_MET.xlsx")


### Part 2: CEH chemical deposition data

The nature of the CEH deposition data required the writing of additional functions and a more convoluted procedure due to it being stored rather oddly as a csv file, despite being in layout a grid.

The coordinates of each data point in the 'grids' were also centroids, whereas opens source geospatial libs (e.g. GDAL) use the top left corner. Hence the 'grid points' had to be shifted prior to rasterisation.

I am unaware of a way of accessing this data via command line/API from the CEH repository so the data would have to be downloaded manually.

Please alter the directory as appropriate.

I saved my deposition data according to the years it covered, hence the structure - below. You will have to do the same.

**Part 2a: The 1986-2012 data**

This [data](https://catalogue.ceh.ac.uk/documents/8e7644fe-9f17-4fc3-8e4e-8b10a42d5d50) has a different set of headings to the latter data (2012-19). The 86-12 headings are:
```python
['easting', 'northing', 'NOx', 'NHx', 'xSOx', 'CaMg', 'year']
```
**If you just want a unified version of both older and newer formats skip to section 2b.**

In [ ]:
depdir = ('DepositionData')

os.chdir(depdir)

dirlist = ['1986-2012', '2013-15', '2017-19']



Here we get a list of files.

In [ ]:
flist = glob(os.path.join(depdir, keywrd))
flist.sort()

list86 = glob(os.path.join(dirlist[0], '*.csv'))

Fix the coordinates for later on. We have files for both forest and moorland assumed landscapes.

In [ ]:
forest = pd.read_csv(list86[2])

moor = pd.read_csv(list86[1])

yrs  = list(forest.year.unique())

# shift from centroid to top left
forest['nx'] = forest['easting']-2500 #back (west)
forest['ny'] = forest['northing']+2500 #up (north)

moor['nx'] = moor['easting']-2500 #back (west)
moor['ny'] = moor['northing']+2500 #up (north)

# create new gdf with shifted coordinates
fgdf = gpd.GeoDataFrame(forest, 
                       geometry=gpd.points_from_xy(forest.nx, forest.ny),

mgdf = gpd.GeoDataFrame(moor, 
                       geometry=gpd.points_from_xy(moor.nx, moor.ny))

fshape = list86[2][:-3]+'shp' # forest

mshape = list86[1][:-3]+'shp' # moor

fgdf.to_file(fshape)

mgdf.to_file(mshape)

We need the raster geo-transform to input into a later function.

In [ ]:
# get the rgt
_, frgt = mt.rasterize_point(fshape, "year=1986", 'H', outras=None,
                     pixel_size=5000, dtype=6)

_, mrgt = mt.rasterize_point(mshape, "year=1986", 'H', outras=None,
                     pixel_size=5000, dtype=6)

Create lists for the subsequent loops to create the array

In [ ]:
chems = fgdf.columns.tolist()[3:13]

list8612f = []
list8612m = []

Here we create a 3-D arrays of the chemical deposition files in year order for each landscape assumption.

In [ ]:
for c in tqdm(chems):
    _,  farray = mt.create_chem_stk(fshape, c,  pixel_size=5000, dtype=6)
    _,  marray = mt.create_chem_stk(mshape, c,  pixel_size=5000, dtype=6)
    
    list8612f.append(farray)
    list8612m.append(marray)

Lastly we can write the data to excel via sampling from the CSS points file.

In [ ]:
gdf = gpd.read_file(inShp) # recall the css file path at the start of the notebook

wb = Workbook()

# Forest
for f, n in tqdm(zip(list8612f, chems)):
    ndf = mt.met_time_series_to_sheet2(gdf, f, frgt, n, yrs)
    values = [ndf.columns] + list(ndf.values)
    ws = wb.new_sheet(n, data=values)

wb.save("SOCD_forest_86-2012.xlsx")

# Moor
for f, n in tqdm(zip(list8612m, chems)):
    ndf = mt.met_time_series_to_sheet2(gdf, f, mrgt, n, yrs)
    values = [ndf.columns] + list(ndf.values)
    ws = wb.new_sheet(n, data=values)

wb.save("SOCD_moor_86-2012.xlsx")

**Section 2b:**

Having done this for the original CEH deposition data (1986-12), we can move onto the later [datasets](https://catalogue.ceh.ac.uk/documents/fd8151e9-0ee2-4dfa-a254-470c9bb9bc1e), which, unfortunately have different headings/formatting. 

Pre 2012 format:
```python
['easting', 'northing', 'NOx', 'NHx', 'xSOx', 'CaMg', 'year']
```

Post 2012 format:
```python
['easting', 'northing', 'nox_f', 'nhx_f', 'nms_f', 'nm_ca_mg_f', 'year']
```
**The following loop merges old and new formats to produce 86-19 depostion files. If we are going to process the 2012-19 files, we are as well to unify the labels and provide a complete file.**


In [ ]:
keywords = ['*/*forest*.csv', '*/*moor*.csv']
outfs = ['ForestComplete.shp', 'MoorComplete.shp']

for k, o in tqdm(zip(keywords, outfs)):

    flist = glob(os.path.join(depdir, k))
    flist.sort()

    dflist = [pd.read_csv(f) for f in flist]

    # First off select & change the columns in the big df to be same as later ones
    # Old format
    #['easting', 'northing', 'NOx', 'NHx', xSOx, 'CaMg', 'year']
    # new format
    #['easting', 'northing', 'nox_f', 'nhx_f', 'nms_f', 'nm_ca_mg_f', 'year']

    dflist[1]['year']=2015
    dflist[2]['year']=2017
    dflist[3]['year']=2019

    dflist[0] = dflist[0][['easting', 'northing', 'NOx', 'NHx', 'xSOx', 'CaMg', 'year']]
    dflist[0].columns = dflist[2].columns

    if k == '*/*forest*.csv':
        #fix the column in no3 - columns (2019) has a format error in it ('nhx_f ')
        dflist[3].columns = dflist[2].columns

    new = pd.concat(dflist)

    new['nx'] = new['easting']-2500 #back (west)
    new['ny'] = new['northing']+2500 #up (north)

    gd = gpd.GeoDataFrame(new, 
                geometry=gpd.points_from_xy(new.nx, new.ny))
    
    gd.set_crs(epsg=27700, inplace=True)
    
    gd.to_file(o)
    


Read in the newly merged files.

In [ ]:
fgdf2 = gpd.read_file(outfs[0])
mgdf2 = gpd.read_file(outfs[1])

Get the information from each to pass into the next cell.

In [ ]:
yrs  = list(fgdf.year.unique())

chemsf = fgdf2.columns.tolist()[3:13]
chemsm = mgdf2.columns.tolist()[3:13]

# get the rgt
_, rgtf = mt.rasterize_point(outfs[0], "year=1986", 'H', outras=None,
                     pixel_size=5000, dtype=6)

_, rgtm = mt.rasterize_point(outfs[1], "year=1986", 'H', outras=None,
                     pixel_size=5000, dtype=6)

Lastly we can write the data to excel via sampling from the CSS points file.

In [ ]:
list19presf = []
list19presm = []


for cf, cm in tqdm(chemsf, chemsm):
    _,  farray = mt.create_chem_stk(outfs[0], cf,  pixel_size=5000, dtype=6)
    _,  marray = mt.create_chem_stk(outfs[1], cm,  pixel_size=5000, dtype=6)
    
    list19presf.append(farray)
    list19presm.append(marray)

# Forest
wb = Workbook()

for f, n in tqdm(zip(list19presf, chemsf)):
    ndf = mt.met_time_series_to_sheet2(gdf, f, rgt, n, yrs)
    values = [ndf.columns] + list(ndf.values)
    ws = wb.new_sheet(n, data=values)

wb.save("SOCD_forest_86-2019.xlsx")

del wb

# Moor
wb = Workbook()

for f, n in tqdm(zip(list19presm, chemsm)):
    ndf = mt.met_time_series_to_sheet2(gdf, f, rgt, n, yrs)
    values = [ndf.columns] + list(ndf.values)
    ws = wb.new_sheet(n, data=values)

wb.save("SOCD_moor_86-2019.xlsx")

### README

It is worth noting that the above does not provide a "readme" sheet. This could be added
at the beginning of a workbook, though it is probably quicker to do this manually. See [here](https://github.com/kz26/PyExcelerate#styling-cells) for further details on formatting if required.

The following will create a quick table of the old and new chemical formats and a hyperlink to the CEH repository in cell 7.


In [42]:
wb = Workbook()

d = [["This dataset is a unification of both old and new CEH deposition headings."],
     ["The old and new values are displayed below."],
     ['easting', 'northing', 'NOx', 'NHx', 'xSOx', 'CaMg', 'year'],
     ['easting', 'northing', 'nox_f', 'nhx_f', 'nms_f', 'nm_ca_mg_f', 'year']] 

ws = wb.new_sheet('README', data=d)

ws.set_cell_style(1, 1, Style(font=Font(bold=True)))
ws.set_cell_style(1, 1, Style(font=Font(underline=True)))

ws.set_cell_style(2, 1, Style(font=Font(bold=True)))
ws.set_cell_style(2, 1, Style(font=Font(underline=True)))

ws.set_cell_value(3, 7, "1986-2012")
ws.set_cell_style(3, 7, Style(font=Font(bold=True)))
ws.set_cell_style(3, 7, Style(font=Font(underline=True)))

ws.set_cell_value(4, 7, "2012-2019")
ws.set_cell_style(4, 7, Style(font=Font(bold=True)))
ws.set_cell_style(4, 7, Style(font=Font(underline=True)))

# can set excel cmds via string
hlink = ('=HYPERLINK("https://catalogue.ceh.ac.uk/documents/fd8151e9-0ee2-4dfa-a254-470c9bb9bc1e",'
         ' "Link to CEH deposition data")')

ws.set_cell_value(6, 1, "An example of the original data can be found here")
ws.set_cell_value(7, 1, hlink)

wb.save('test.xlsx')
